# Getting Rain Data from Dark Sky

In [1]:
!ls

AccessingDarkSkyAPI.ipynb README.md                 keys.py
CONTRIBUTING.md           __pycache__
LICENSE.md                database.sqlite


In [2]:
# Import requests to access the API
import requests
# Pandas because doesn't everyone love pandas?
import pandas as pd
# Our data is stored in an SQL database, thus
import sqlite3

# I have my API key saved in a python file, thus
from keys import ds_api_key as ds_key
# Remember, you can also have it as a json in some other folder
# BUT NEVER JUST PASTE YOUR API KEY. NEVER EVER.

In [3]:
# Connecting to the sql database that's in this repo
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [4]:
# Here, I'm just grabbing the dates for matches in the 2011 season
cur.execute("""SELECT Date 
               FROM Matches
               WHERE Season = "2011";""")

In [5]:
# Turning the response from my sql query into a dataframe
df = pd.DataFrame(cur.fetchall())

In [6]:
# Renaming my column and checking this out
df.rename(columns={0:"Date"}, inplace=True)
df.head()

,Date
0,2012-03-31
1,2011-12-11
2,2011-08-13
3,2011-11-27
4,2012-02-18


In [7]:
# How big is this dataframe?
df.size

992

In [8]:
# Creating a list object of the dates
dates = list(df["Date"])
# Sanity check: this should be the same length as the df
len(dates)

992

In [9]:
# Method 1 for finding unique dates
unique_dates = set(dates)
len(unique_dates)

165

In [10]:
# Method 2 for finding unique dates
unique_dates_array = df["Date"].unique()
len(unique_dates_array)

165

In [11]:
# Let's go ahead and define our Berlin geographic data
lat = "52.5200"
long = "13.4050"

In [12]:
# And now, let's test our API request
# Defining a test date to test with
testdate = "2011-11-27"
# Our URL comes from playing around in Postman, to get the params we want
url = f'https://api.darksky.net/forecast/{ds_key}/{lat},{long},{testdate}T20:00:00?exclude=currently,hourly,flags'
# Requesting a response from our URL, and going ahead and getting it to json
r_test = requests.get(url).json()

In [13]:
# Let's check our test response out
r_test

{'latitude': 52.52,
 'longitude': 13.405,
 'timezone': 'Europe/Berlin',
 'daily': {'data': [{'time': 1322348400,
    'summary': 'Windy in the evening.',
    'icon': 'wind',
    'sunriseTime': 1322376600,
    'sunsetTime': 1322406060,
    'moonPhase': 0.1,
    'precipIntensity': 0,
    'precipIntensityMax': 0,
    'precipProbability': 0,
    'temperatureHigh': 50.66,
    'temperatureHighTime': 1322408340,
    'temperatureLow': 39.26,
    'temperatureLowTime': 1322463600,
    'apparentTemperatureHigh': 50.77,
    'apparentTemperatureHighTime': 1322408400,
    'apparentTemperatureLow': 33.68,
    'apparentTemperatureLowTime': 1322463600,
    'dewPoint': 40.06,
    'humidity': 0.73,
    'windSpeed': 19.56,
    'windGust': 41.05,
    'windGustTime': 1322412420,
    'windBearing': 243,
    'cloudCover': 0.71,
    'uvIndex': 1,
    'uvIndexTime': 1322391300,
    'visibility': 6.216,
    'temperatureMin': 45.95,
    'temperatureMinTime': 1322348400,
    'temperatureMax': 50.66,
    'temperatur

In [14]:
# If we want to get down to 'precipIntensity', which always is part of the 
# response, how do we do that? Iteratively!
# First, get one level down, to daily
r_test["daily"]

{'data': [{'time': 1322348400,
   'summary': 'Windy in the evening.',
   'icon': 'wind',
   'sunriseTime': 1322376600,
   'sunsetTime': 1322406060,
   'moonPhase': 0.1,
   'precipIntensity': 0,
   'precipIntensityMax': 0,
   'precipProbability': 0,
   'temperatureHigh': 50.66,
   'temperatureHighTime': 1322408340,
   'temperatureLow': 39.26,
   'temperatureLowTime': 1322463600,
   'apparentTemperatureHigh': 50.77,
   'apparentTemperatureHighTime': 1322408400,
   'apparentTemperatureLow': 33.68,
   'apparentTemperatureLowTime': 1322463600,
   'dewPoint': 40.06,
   'humidity': 0.73,
   'windSpeed': 19.56,
   'windGust': 41.05,
   'windGustTime': 1322412420,
   'windBearing': 243,
   'cloudCover': 0.71,
   'uvIndex': 1,
   'uvIndexTime': 1322391300,
   'visibility': 6.216,
   'temperatureMin': 45.95,
   'temperatureMinTime': 1322348400,
   'temperatureMax': 50.66,
   'temperatureMaxTime': 1322408340,
   'apparentTemperatureMin': 39.83,
   'apparentTemperatureMinTime': 1322348400,
   'appa

In [15]:
# Next level down
r_test["daily"]["data"]

[{'time': 1322348400,
  'summary': 'Windy in the evening.',
  'icon': 'wind',
  'sunriseTime': 1322376600,
  'sunsetTime': 1322406060,
  'moonPhase': 0.1,
  'precipIntensity': 0,
  'precipIntensityMax': 0,
  'precipProbability': 0,
  'temperatureHigh': 50.66,
  'temperatureHighTime': 1322408340,
  'temperatureLow': 39.26,
  'temperatureLowTime': 1322463600,
  'apparentTemperatureHigh': 50.77,
  'apparentTemperatureHighTime': 1322408400,
  'apparentTemperatureLow': 33.68,
  'apparentTemperatureLowTime': 1322463600,
  'dewPoint': 40.06,
  'humidity': 0.73,
  'windSpeed': 19.56,
  'windGust': 41.05,
  'windGustTime': 1322412420,
  'windBearing': 243,
  'cloudCover': 0.71,
  'uvIndex': 1,
  'uvIndexTime': 1322391300,
  'visibility': 6.216,
  'temperatureMin': 45.95,
  'temperatureMinTime': 1322348400,
  'temperatureMax': 50.66,
  'temperatureMaxTime': 1322408340,
  'apparentTemperatureMin': 39.83,
  'apparentTemperatureMinTime': 1322348400,
  'apparentTemperatureMax': 50.77,
  'apparentTem

In [16]:
# The next level is a list, and we want the first element of it, so
r_test["daily"]["data"][0]

{'time': 1322348400,
 'summary': 'Windy in the evening.',
 'icon': 'wind',
 'sunriseTime': 1322376600,
 'sunsetTime': 1322406060,
 'moonPhase': 0.1,
 'precipIntensity': 0,
 'precipIntensityMax': 0,
 'precipProbability': 0,
 'temperatureHigh': 50.66,
 'temperatureHighTime': 1322408340,
 'temperatureLow': 39.26,
 'temperatureLowTime': 1322463600,
 'apparentTemperatureHigh': 50.77,
 'apparentTemperatureHighTime': 1322408400,
 'apparentTemperatureLow': 33.68,
 'apparentTemperatureLowTime': 1322463600,
 'dewPoint': 40.06,
 'humidity': 0.73,
 'windSpeed': 19.56,
 'windGust': 41.05,
 'windGustTime': 1322412420,
 'windBearing': 243,
 'cloudCover': 0.71,
 'uvIndex': 1,
 'uvIndexTime': 1322391300,
 'visibility': 6.216,
 'temperatureMin': 45.95,
 'temperatureMinTime': 1322348400,
 'temperatureMax': 50.66,
 'temperatureMaxTime': 1322408340,
 'apparentTemperatureMin': 39.83,
 'apparentTemperatureMinTime': 1322348400,
 'apparentTemperatureMax': 50.77,
 'apparentTemperatureMaxTime': 1322408400}

In [17]:
# Now, let's get to precipIntensity!
r_test["daily"]["data"][0]["precipIntensity"]
# On our test date, it didn't rain - no precipIntensity

0

## Option 1: Use a DataFrame Throughout

In [18]:
# Creating a dataframe just of our unique dates
unique_date_df = pd.DataFrame(unique_dates, columns=["Unique Date"])
unique_date_df.head()

,Unique Date
0,2012-04-09
1,2012-03-04
2,2012-01-21
3,2012-03-31
4,2012-02-03


In [19]:
# Creating a new column for weather, easier to do it here
unique_date_df["Weather"] = "not filled"
unique_date_df.head()

,Unique Date,Weather
0,2012-04-09,not filled
1,2012-03-04,not filled
2,2012-01-21,not filled
3,2012-03-31,not filled
4,2012-02-03,not filled


In [20]:
# Creating a sample of our df for this first way of doing this
sample1 = unique_date_df.head(20)

In [21]:
sample1.head()

,Unique Date,Weather
0,2012-04-09,not filled
1,2012-03-04,not filled
2,2012-01-21,not filled
3,2012-03-31,not filled
4,2012-02-03,not filled


In [22]:
# Alright! Let's test our API request with our sample1

# Creating a for-loop to iterate over the range of our sample
for row in range(len(sample1)):
    
    # Grabbing the date for each row as its own variable
    date = sample1["Unique Date"][row]
    # Creating our url based on Postman, and our tests above
    url = f'https://api.darksky.net/forecast/{ds_key}/{lat},{long},{date}T20:00:00?exclude=currently,hourly,flags'
    response = requests.get(url)
    
    # Here's a nice if/else statement to check the status code of our request
    if response.status_code == 200:
        # Now, getting the response as a json/dict
        data = response.json()
        
        # Here, we're creating an if/else statement to check whether there was
        # any precipitation, and if there was precip I'll get the type
        if data["daily"]["data"][0]['precipIntensity'] == 0:
            # So, if there was no precipitation, our weather will say this
            sample1["Weather"][row] = "no precip"
        else:
            # And if there was precipitation, I'll grab the icon as my result
            result = data["daily"]["data"][0]["icon"]
            sample1["Weather"][row] = result
            
    else:
        print('Hit an error.')
# Just ignore this lovely SettingWithCopy warning

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
# Let's see if this worked!
sample1

,Unique Date,Weather
0,2012-04-09,no precip
1,2012-03-04,no precip
2,2012-01-21,no precip
3,2012-03-31,rain
4,2012-02-03,no precip
5,2011-09-12,rain
6,2012-01-31,no precip
7,2011-09-17,no precip
8,2011-09-19,rain
9,2011-12-26,rain


In [24]:
# Now, since the sample worked, I can do it for my whole unique date df
        
# This code is the same as the above, just with this df instead of sample1
for row in range(len(unique_date_df)):
    date = unique_date_df["Unique Date"][row]
    url = f'https://api.darksky.net/forecast/{ds_key}/{lat},{long},{date}T20:00:00?exclude=currently,hourly,flags'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if data["daily"]["data"][0]['precipIntensity'] == 0:
            unique_date_df["Weather"][row] = "no precip"
        else:
            result = data["daily"]["data"][0]["icon"]
            unique_date_df["Weather"][row] = result
            
    else:
        print('Hit an error.')

In [27]:
# Let's check to make sure there's no more 'not filled' rows
unique_date_df["Weather"].value_counts()

no precip    107
rain          57
snow           1
Name: Weather, dtype: int64

Now what? You can join this `unique_date_df` to a matches dataframe or wherever else you're trying to bring in your rain data!

## Option 2: Use a Dictionary

In [31]:
# We already have an array of our unique dates
unique_dates_array[:10]

array(['2012-03-31', '2011-12-11', '2011-08-13', '2011-11-27',
       '2012-02-18', '2012-01-20', '2012-02-04', '2012-04-21',
       '2011-09-18', '2011-10-23'], dtype=object)

In [32]:
# Let's do a sample for this second option
sample2 = unique_dates_array[:20]

In [33]:
sample2

array(['2012-03-31', '2011-12-11', '2011-08-13', '2011-11-27',
       '2012-02-18', '2012-01-20', '2012-02-04', '2012-04-21',
       '2011-09-18', '2011-10-23', '2011-10-01', '2012-03-03',
       '2011-08-27', '2012-03-17', '2011-11-06', '2012-05-05',
       '2012-04-11', '2011-12-17', '2012-02-03', '2011-10-29'],
      dtype=object)

In [37]:
# First, let's do this on our sample!

# Defining an empty dictionary to populate
sample_dict = {}

# Iterating over our sample array
for date in sample2:
    # Again, using the date from above
    url = f'https://api.darksky.net/forecast/{ds_key}/{lat},{long},{date}T20:00:00?exclude=currently,hourly,flags'
    
    # I did not write that if/else statement to check the status, but you could
    # Here is the request response
    r = requests.get(url).json()
    
    # Defining a variable for the result of the precipIntensity
    precipInt = r["daily"]["data"][0]["precipIntensity"]
    
    # Now, I'm using an if/elif statement to check whether there was any precip
    if precipInt == 0:
        # If there's no precipitation, the value of the date will be this
        sample_dict[date] = "no precip"
    # Could also just write else here    
    elif precipInt != 0:
        # If there is precipitation, the value of the date will be the type
        sample_dict[date] = r["daily"]["data"][0]["precipType"]

In [38]:
# Sanity check - it worked!
sample_dict

{'2012-03-31': 'rain',
 '2011-12-11': 'no precip',
 '2011-08-13': 'no precip',
 '2011-11-27': 'no precip',
 '2012-02-18': 'no precip',
 '2012-01-20': 'rain',
 '2012-02-04': 'no precip',
 '2012-04-21': 'no precip',
 '2011-09-18': 'rain',
 '2011-10-23': 'rain',
 '2011-10-01': 'no precip',
 '2012-03-03': 'no precip',
 '2011-08-27': 'rain',
 '2012-03-17': 'no precip',
 '2011-11-06': 'rain',
 '2012-05-05': 'rain',
 '2012-04-11': 'no precip',
 '2011-12-17': 'rain',
 '2012-02-03': 'no precip',
 '2011-10-29': 'no precip'}

In [40]:
# Since the sample worked, let's do it for all unique dates
# New dictionary here
precip_dict = {}

# The rest of this is the same code
for date in unique_dates_array:
    url = f'https://api.darksky.net/forecast/{ds_key}/{lat},{long},{date}T20:00:00?exclude=currently,hourly,flags'
    
    r = requests.get(url).json()

    precipInt = r["daily"]["data"][0]["precipIntensity"]
    
    if precipInt == 0:
        precip_dict[date] = "no precip"
    elif precipInt != 0:
        precip_dict[date] = r["daily"]["data"][0]["precipType"]

In [43]:
precip_dict

{'2012-03-31': 'rain',
 '2011-12-11': 'no precip',
 '2011-08-13': 'no precip',
 '2011-11-27': 'no precip',
 '2012-02-18': 'no precip',
 '2012-01-20': 'rain',
 '2012-02-04': 'no precip',
 '2012-04-21': 'no precip',
 '2011-09-18': 'rain',
 '2011-10-23': 'rain',
 '2011-10-01': 'no precip',
 '2012-03-03': 'no precip',
 '2011-08-27': 'rain',
 '2012-03-17': 'no precip',
 '2011-11-06': 'rain',
 '2012-05-05': 'rain',
 '2012-04-11': 'no precip',
 '2011-12-17': 'rain',
 '2012-02-03': 'no precip',
 '2011-10-29': 'no precip',
 '2012-01-22': 'rain',
 '2011-12-03': 'rain',
 '2012-04-14': 'no precip',
 '2012-03-25': 'no precip',
 '2012-03-10': 'no precip',
 '2012-04-07': 'no precip',
 '2011-11-19': 'no precip',
 '2011-10-14': 'no precip',
 '2011-09-24': 'no precip',
 '2012-04-28': 'no precip',
 '2011-12-18': 'rain',
 '2012-03-02': 'no precip',
 '2012-03-16': 'no precip',
 '2012-02-17': 'rain',
 '2011-08-06': 'rain',
 '2011-11-04': 'no precip',
 '2011-09-16': 'no precip',
 '2011-07-15': 'no precip',
 

Now what?

In [ ]:
# I can now change this to a dataframe to do a join with some other df
rain_df = pd.DataFrame(list(precip_dict.items()), columns=['Date', 'Precip'])
rain_df.head()

But! I can also use a dictionary and iterate over the dictionary plus wherever I have my match data, to get the precipitation data incorporated into the rest of my data. Ask me how if you want help with that!